# Creating Lab-Wide Relational Database
This is a script written by Zach Monge and is being used to create a lab-wide relational database. Right now it is just being used for one study, which is called ERDeg. To create the database I used SQLite and specifically sqlite3, which is the Python implementation of SQLite. I am not able to make the data publically available because it contains participants' personal information. <br><br>
The diagraph of the database may be found in the repository.

## Importing functions and creating database

In [1]:
#Importing functions

import sqlite3
import pandas as pd
import pdb

In [2]:
PATH_data='Z:\\ERMatchOA.02\\Data\\SQL_upload'

Creating database

In [3]:
conn = sqlite3.connect(f'{PATH_data}\\ERDeg_database.sqlite')
cur = conn.cursor()

## Creating Tables

Creating Participant table. Each participant is assigned an id, which is determined from unique entries of first name, last name, and date of birth.

In [7]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS Participant
            (id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT,
            dob DATE, phone TEXT, email TEXT, gender TEXT, education FLOAT, 
            UNIQUE(first_name, last_name, dob))
            ''')

Create Study table. Each study is assigned an id.

In [8]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS Study
            (id INTEGER PRIMARY KEY, study_name TEXT UNIQUE)
            ''')

Create ScreeningQuestionnaire table. Only one entry is allowed for each participant in a study.

In [9]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS ScreeningQuestionnaire
            (participant_id INTEGER, study_id INTEGER, assessment_date DATE,
            moca INTEGER, bdi INTEGER, vviq INTEGER, UNIQUE(participant_id, study_id))
            ''')

Create ScanInformation table. Only one entry is allowed for each participant in a study.

In [10]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS ScanInformation
            (participant_id INTEGER, study_id INTEGER,
            scan_id1 TEXT, scan_id2 TEXT, UNIQUE(participant_id, study_id))
            ''')

Create StudySpecificId table. Only one entry is allowed for each participant in a study.

In [11]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS StudySpecificId
            (participant_id INTEGER, study_id INTEGER,
            participant_study_id INTEGER, UNIQUE(participant_id, study_id))
            ''')

Create NihToolboxTest table. Each test is assigned an id.

In [12]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS NihToolboxTest
            (id INTEGER PRIMARY KEY, test_name TEXT UNIQUE)
            ''')

Create NihToolboxScore table. Only one entry is allowed for each test for each participant in a study.

In [13]:
cur.execute('''
            CREATE TABLE IF NOT EXISTS NihToolboxScore
            (participant_id INTEGER, study_id INTEGER, date_time DATETIME,
            test_id INTEGER, raw_score INTEGER, theta FLOAT, se FLOAT,
            itm_cnt INTEGER, computed_score FLOAT, uncorrected_standard_score INTEGER,
            age_corrected_score INTEGER, national_percentile INTEGER, t_score INTEGER,
            UNIQUE(participant_id, study_id, test_id))
            ''')

## Inserting data

Below reading in CSV that contains participant information. Again, I am not able to show any of the data because it contains personal information, but you can see the column names below.

In [17]:
demographics_df = pd.read_csv(f'{PATH_data}\\ERDeg_Demographics_SQL.csv')
demographics_df.columns

Index(['id', 'name', 'scanid_1', 'scanid_2', 'age_grp', 'gender', 'phone',
       'email', 'task_version', 'DOB', 'age', 'education', 'assessment_date',
       'MoCA', 'BDI', 'VVIQ'],
      dtype='object')

Inserting data for Participant table

In [18]:
# Inserting new participants
for row in demographics_df.iterrows():
    row=row[1] # Getting data for one row
    
    # Inserting new participants
    cur.execute('''INSERT OR IGNORE INTO Participant (first_name, last_name, dob) VALUES (?, ?, ?)''',
           (row['name'].split()[0], row['name'].split()[-1], row['DOB']))
    
conn.commit()

# Inserting other data on participants contained in Participant table
for row in demographics_df.iterrows():
    row=row[1] # Getting data for one row
    
    # Getting id number
    cur.execute('SELECT id FROM Participant WHERE first_name = ? AND last_name = ? AND dob = ? LIMIT 1',
               (row['name'].split()[0], row['name'].split()[-1], row['DOB']))
    id_num = cur.fetchone()[0]
    
    # Updating
    cur.execute('''UPDATE Participant SET phone=?, email=?, gender=?, education=? WHERE id=?''',
               (row['phone'], row['email'], row['gender'], row['education'], id_num))
    
conn.commit()

Inserting data for Study table. Again there is only one study here so far which is called *ERDeg*.

In [19]:
cur.execute('''INSERT OR IGNORE INTO Study (study_name) VALUES (?)''',
           ('ERDeg',))
conn.commit()

Inserting data into ScreeningQuestionnaire, ScanInformation, and StudySpecificId table

In [20]:
for row in demographics_df.iterrows():
    row=row[1] # Getting data for one row 
    
    # Getting participant id number
    cur.execute('SELECT id FROM Participant WHERE first_name = ? AND last_name = ? AND dob = ? LIMIT 1',
               (row['name'].split()[0], row['name'].split()[-1], row['DOB']))
    participant_id = cur.fetchone()[0]
    
    # Getting study id number for ERDeg
    cur.execute('''SELECT id FROM Study WHERE study_name = ? LIMIT 1''',
               ('ERDeg',))
    study_id = cur.fetchone()[0]
    
    # Inserting data into ScreeningQuestionnaire table
    cur.execute('''INSERT OR IGNORE INTO ScreeningQuestionnaire (participant_id,
                study_id, assessment_date, moca, bdi, vviq) VALUES (?,?,?,?,?,?)''',
               (participant_id, study_id, row['assessment_date'], row['MoCA'],
               row['BDI'], row['VVIQ']))
    
    # Inserting data into ScanInformation table
    cur.execute('''INSERT OR IGNORE INTO ScanInformation (participant_id,
    study_id, scan_id1, scan_id2) VALUES (?,?,?,?)''',
               (participant_id, study_id, row['scanid_1'], row['scanid_2']))
    
    # Inserting data in StudySpecificId table
    cur.execute('''INSERT OR IGNORE INTO StudySpecificId (participant_id,
    study_id, participant_study_id) VALUES (?,?,?)''',
               (participant_id,study_id,row['id']))

conn.commit()

Inserting data into NihToolboxTest

In [21]:
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Picture Vocabulary Test Age 3+ v2.0')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Flanker Inhibitory Control and Attention Test Age 12+ v2.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox List Sorting Working Memory Test Age 7+ v2.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Pattern Comparison Processing Speed Test Age 7+ v2.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Picture Sequence Memory Test Age 8+ Form A v2.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Picture Vocabulary Test Age 3+ Practice v2.0')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Oral Reading Recognition Test Age 3+ v2.0')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('Cognition Fluid Composite v1.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('Cognition Crystallized Composite v1.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('Cognition Total Composite Score v1.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('Cognition Early Childhood Composite v1.1')''')
cur.execute('''INSERT OR IGNORE INTO NihToolboxTest (test_name) VALUES ('NIH Toolbox Auditory Verbal Learning Test (Rey) 8+ v2.0')''')
conn.commit()

Import NIH Toolbox Scores

In [23]:
nih_data_df=pd.read_csv("Z:\\ERMatchOA.02\\Data\\NIH_Toolbox\\ERDeg_NIHToolbox_SQL_upload.csv")
nih_data_df.columns

Index(['PIN', 'DeviceID', 'Assessment Name', 'Inst', 'RawScore', 'Theta',
       'TScore', 'SE', 'ItmCnt', 'DateFinished', 'Column1', 'Column2',
       'Column3', 'Column4', 'Column5', 'Language', 'Computed Score',
       'Uncorrected Standard Score', 'Age-Corrected Standard Score',
       'National Percentile (age adjusted)', 'Fully-Corrected T-score',
       'InstrumentBreakoff', 'InstrumentStatus2', 'InstrumentRCReason',
       'InstrumentRCReasonOther', 'App Version', 'iPad Version',
       'Firmware Version'],
      dtype='object')

Importing data into NihToolboxScore Table

In [24]:
# Getting study_id for ERDeg
cur.execute('''SELECT id FROM Study WHERE study_name = "ERDeg" LIMIT 1''')
study_id = cur.fetchone()[0]

for row in nih_data_df.iterrows():
    row=row[1] # Getting data for one row
    
    
    # Getting participant_id
    cur.execute('''SELECT participant_id FROM StudySpecificId WHERE study_id = ? AND participant_study_id = ? LIMIT 1''',
               (study_id, row['PIN'][-4:]))
    participant_id = cur.fetchone()[0]
    
    # Getting test ID number
    cur.execute('''SELECT id FROM NihToolboxTest WHERE test_name = ? LIMIT 1''',
               (row['Inst'], ))
    test_id = cur.fetchone()[0]

    # Inserting scores
    cur.execute('''INSERT OR IGNORE INTO NihToolboxScore (participant_id,study_id,date_time,
    test_id, raw_score,theta,se,itm_cnt,computed_score,uncorrected_standard_score,age_corrected_score,
    national_percentile,t_score) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)''',
               (participant_id,study_id,row['DateFinished'],test_id,row['RawScore'],
               row['Theta'],row['SE'],row['ItmCnt'],row['Computed Score'],
               row['Uncorrected Standard Score'],row['Age-Corrected Standard Score'],
               row['National Percentile (age adjusted)'], row['Fully-Corrected T-score']))
    
conn.commit()

In [25]:
cur.close()